# ML TempCNN Classification – Sentinel-2 + Training Geometries

This example demonstrates an openEO-style ML pipeline using a
Temporal CNN (TempCNN) classifier built on PyTorch:

1. Load Sentinel-2 L2A data cube
2. Load training polygons (GeoJSON) with labels
3. Aggregate temporally (yearly median)
4. Compute NDVI as additional band
5. Extract zonal statistics per training polygon via `aggregate_spatial`
6. Train a TempCNN classifier
7. Apply predictions to the full raster
8. Optionally save as GeoTIFF

In [1]:
from openeo_core import DataCube
from openeo_core.model import mlm_class_tempcnn, ml_fit, ml_predict, save_ml_model
import geopandas as gpd

In [2]:
# Load Sentinel-2 L2A data cube (Brittany, France – overlaps with breizh_data)
datacube = DataCube.load_collection(
    "sentinel-2-l2a",
    spatial_extent={"west": -3.8, "south": 48.2, "east": -3.6, "north": 48.4, "crs": 4326},
    temporal_extent=("2022-01-01", "2022-12-31"),
    bands=["blue", "green", "red", "rededge1", "rededge2", "rededge3", "nir"],
    properties={"eo:cloud_cover": {"lt": 50}},
    max_items=50,
)

In [3]:
datacube.data

<xarray.DataArray 'stackstac-1d6b96edd93777eb593a359638ebb35e' (time: 50,
                                                                bands: 7,
                                                                latitude: 3357,
                                                                longitude: 1486)> Size: 14GB
dask.array<fetch_raster_window, shape=(50, 7, 3357, 1486), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/52)
  * time                                     (time) datetime64[ns] 400B 2022-...
    id                                       (time) <U24 5kB 'S2A_30UVU_20220...
  * bands                                    (bands) <U8 224B 'blue' ... 'nir'
  * longitude                                (longitude) float64 12kB -3.8 .....
  * latitude                                 (latitude) float64 27kB 48.4 ......
    mgrs:grid_square                         <U2 8B 'VU'
    ...                                       ...
    s2:tile_id                               (time) object 400B None ... None
    view:azimuth                             (time) object 400B None ... None
    proj:centroid                            (time) object 400B None ... None
    view:incidence_angle                     (time) object 400B None ... None
    gsd                                      (bands) int64 56B 10 10 ... 20 10
    epsg                                     int64 8B 4326
Attributes:
    spec:           RasterSpec(epsg=4326, bounds=(-3.8000325594021027, 48.199...
    crs:            epsg:4326
    transform:      | 0.00, 0.00,-3.80|\n| 0.00,-0.00, 48.40|\n| 0.00, 0.00, ...
    resolution_xy:  (0.00013461449429317732, 5.9592606233034484e-05)

In [4]:
# Load training data (Breizh crop parcels with class_id as label)
training_path = "./input/breizh_data.geojson"
gdf = gpd.read_file(training_path)
# Limit to polygons overlapping the bbox for faster execution
from shapely.geometry import box
bbox = (-3.8, 48.2, -3.6, 48.4)
clip_box = gpd.GeoDataFrame(geometry=[box(*bbox)], crs=4326)
gdf = gpd.clip(gdf, clip_box).head(100)
gdf = gdf.to_crs(4326)

In [5]:
# Aggregate along the temporal dimension (yearly median for simpler features)
datacube = datacube.aggregate_temporal(period="year", reducer="median")

In [6]:
# Calculate NDVI as an additional band
datacube = datacube.ndvi(red="red", nir="nir", target_band="NDVI")

In [7]:
datacube.data

<xarray.DataArray 'stackstac-1d6b96edd93777eb593a359638ebb35e' (time: 1,
                                                                bands: 8,
                                                                latitude: 3357,
                                                                longitude: 1486)> Size: 319MB
dask.array<concatenate, shape=(1, 8, 3357, 1486), dtype=float64, chunksize=(1, 1, 680, 493), chunktype=numpy.ndarray>
Coordinates: (12/16)
  * bands                                    (bands) object 64B 'blue' ... 'N...
  * longitude                                (longitude) float64 12kB -3.8 .....
  * latitude                                 (latitude) float64 27kB 48.4 ......
    mgrs:grid_square                         <U2 8B 'VU'
    s2:product_type                          <U7 28B 'S2MSI2A'
    constellation                            <U10 40B 'sentinel-2'
    ...                                       ...
    mgrs:utm_zone                            int64 8B 30
    grid:code                                <U10 40B 'MGRS-30UVU'
    s2:saturated_defective_pixel_percentage  int64 8B 0
    gsd                                      (bands) int64 64B 10 10 ... 10 10
    epsg                                     int64 8B 4326
  * time                                     (time) <U4 16B '2022'
Attributes:
    spec:           RasterSpec(epsg=4326, bounds=(-3.8000325594021027, 48.199...
    crs:            epsg:4326
    transform:      | 0.00, 0.00,-3.80|\n| 0.00,-0.00, 48.40|\n| 0.00, 0.00, ...
    resolution_xy:  (0.00013461449429317732, 5.9592606233034484e-05)

In [8]:
# Aggregate statistics for each training geometry (median per band per month)
# Returns a GeoDataFrame with band values as columns + preserved class_id
training_data = datacube.aggregate_spatial(geometries=gdf, reducer="median")

/Users/brianpondi/.venvs/venv-openeo/lib/python3.10/site-packages/dask/_task_spec.py:767: RuntimeWarning: All-NaN slice encountered
  return self.func(*new_argspec, **kwargs)
/Users/brianpondi/.venvs/venv-openeo/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:1217: RuntimeWarning: All-NaN slice encountered
  return fnb._ureduce(a, func=_nanmedian, keepdims=keepdims,


In [9]:
# training_data is a DataCube wrapping a GeoDataFrame; extract for ml_fit
# Use only band-derived columns (exclude GeoJSON metadata). aggregate_spatial may use
# time-suffixed names (e.g. red_2022-12) when temporal dim exists; map to band order.
band_names = list(datacube.data.coords["bands"].values)
available = training_data.data.columns
feature_cols = []
for b in band_names:
    matches = [c for c in available if c == b or str(c).startswith(f"{b}_")]
    if matches:
        feature_cols.append(matches[0])
    else:
        raise KeyError(f"Band {b!r} not found in training columns: {list(available)}")
training_gdf = training_data.data[feature_cols].copy()
training_gdf.columns = band_names  # align with prediction cube
training_gdf["label"] = training_data.data["class_id"].astype(int)
training_gdf = gpd.GeoDataFrame(training_gdf, geometry=training_data.data.geometry, crs=training_data.data.crs)

In [10]:
# Initialize the TempCNN model and train it
tempcnn_model = mlm_class_tempcnn(
    cnn_layers=[64, 64, 64],
    cnn_kernels=[5, 5, 5],
    cnn_dropout_rates=[0.2, 0.2, 0.2],
    dense_layer_nodes=128,
    epochs=50,
    batch_size=32,
    learning_rate=0.001,
    seed=42,
)
tempcnn_model = ml_fit(tempcnn_model, training_set=training_gdf, target="label")

In [15]:
save_ml_model(tempcnn_model, "./tempcnn_model_breizh")

True

In [12]:
# Apply the trained model to the data cube
predictions = ml_predict(datacube.data, tempcnn_model)

In [13]:
# Inspect the prediction cube
predictions

<xarray.DataArray '_predict_block-7a06b5d790f81970d168f36198ee2d89' (
                                                                     predictions: 1,
                                                                     time: 1,
                                                                     latitude: 3357,
                                                                     longitude: 1486)> Size: 40MB
dask.array<getitem, shape=(1, 1, 3357, 1486), dtype=float64, chunksize=(1, 1, 170, 1486), chunktype=numpy.ndarray>
Coordinates: (12/15)
  * predictions                              (predictions) object 8B '0'
  * time                                     (time) object 8B '2022'
  * latitude                                 (latitude) float64 27kB 48.4 ......
  * longitude                                (longitude) float64 12kB -3.8 .....
    mgrs:grid_square                         <U2 8B 'VU'
    s2:product_type                          <U7 28B 'S2MSI2A'
    ...                                       ...
    s2:datatake_type                         <U8 32B 'INS-NOBS'
    proj:code                                <U10 40B 'EPSG:32630'
    mgrs:utm_zone                            int64 8B 30
    grid:code                                <U10 40B 'MGRS-30UVU'
    s2:saturated_defective_pixel_percentage  int64 8B 0
    epsg                                     int64 8B 4326

In [16]:
# Optional: Save as GeoTIFF
try:
    out_path = "./output/predictions_tempcnn.tif"
    pred_da = predictions.sel(predictions="0").squeeze()
    if "latitude" in pred_da.dims and "longitude" in pred_da.dims:
        pred_da.rio.to_raster(out_path)
        print(f"Saved to {out_path}")
except Exception as e:
    print(f"Save skipped: {e}")

Saved to ./output/predictions_tempcnn.tif
